In [33]:
import json

import pandas as pd
import seaborn as sns
from datasets import load_dataset
from langchain_core.utils.json import parse_json_markdown

sns.set_theme()

In [22]:
ds = load_dataset("data/datasets/pl/swiss_franc_loans_annotated", split="test")
df = ds.to_pandas()
df.head()

,context,output
0,Sygn. akt XV C 1471/12\n\nWYROK\nW IMIENIU RZE...,"{""sprawa_frankowiczow"":""Nie"",""apelacja"":null,""..."
1,Sygn. akt V Ca 2924/12\n\nWYROK\nW IMIENIU RZE...,"{""sprawa_frankowiczow"":""Tak"",""apelacja"":""warsz..."
2,Sygn. akt I C 376/13\n\nWYROK\nW IMIENIU RZECZ...,"{""sprawa_frankowiczow"":""Tak"",""apelacja"":null,""..."
3,Sygn. akt I ACa 801/13\n\nWYROK\nW IMIENIU RZE...,"{""sprawa_frankowiczow"":""Tak"",""apelacja"":""Biały..."
4,Sygn. akt I ACa 812/13\n\nWYROK\nW IMIENIU RZE...,"{""sprawa_frankowiczow"":""Tak"",""apelacja"":""Gdańs..."


# Fix values

In [23]:
df["output"] = df["output"].apply(parse_json_markdown)

In [26]:
apelacja_map = {
    "warszawska": "warszawska",
    "poznańska": "poznańska",
    "białystok": "białostocka",
    "katowice": "katowicka",
    "gdańska": "gdańska",
    "krakowska": "krakowska",
    "szczecin": "szczecińska",
    "katowicka": "katowicka",
    "szczecińska": "szczecińska",
    "olsztyńska": "olsztyńska",
    "warszawa": "warszawska",
    "olsztyn": "olsztyńska",
    "łódź": "łódzka",
    "białostocka": "białostocka",
    "gdańsk": "gdańska",
    "wrocławska": "wrocławska",
    "kraków": "krakowska",
    "łódzka": "łódzka",
    "lodzka": "łódzka",
    "bydgoszcz": "bydgoska",
    "lubelska": "lubelska",
    "lublin": "lubelska",
    "bydgoska": "bydgoska",
}


def _map_apelacja(x):
    if x["apelacja"] is not None:
        x["apelacja"] = apelacja_map[x["apelacja"].lower().strip()]
    return x


df["output"] = df["output"].apply(_map_apelacja)

In [36]:
store_df = df.copy()
store_df["output"] = store_df["output"].apply(json.dumps)
store_df.to_json("data/datasets/pl/swiss_franc_loans_annotated/test.json", orient="records", index=False, indent=4)

In [27]:
out = pd.DataFrame(df["output"].to_list())
out.head()

,sprawa_frankowiczow,apelacja,data_wyroku,typ_sadu,instancja_sadu,podstawa_prawna,podstawa_prawna_podana,rodzaj_roszczenia,modyfikacje_powodztwa,typ_modyfikacji,...,odsetki_ustawowe,data_rozpoczecia_odsetek,koszty_postepowania,beneficjent_kosztow,zabezpieczenie_udzielone,rodzaj_zabezpieczenia,zabezpieczenie_pierwsza_instancja,czas_trwania_sprawy,wynik_sprawy,szczegoly_wyniku_sprawy
0,Nie,None,2013-04-18,Sąd Okręgowy,Sąd I instancji,"art. 376 §1 k.c., art. 6 k.c.",Tak,O zapłatę,Tak,Kwoty roszczenia,...,Nie,None,Nie,None,Nie było wniosku o zabezpieczenie,None,None,None,Wygrana banku,"Powództwo oddalone w całości, sąd odstąpił od ..."
1,Tak,warszawska,2013-05-20,Sąd Okręgowy,Sąd odwoławczy,"art. 385^1 § 1 k.c., art. 385^2 k.c., art. 6 k...",Tak,O zapłatę,Nie,None,...,Tak,Od dnia wezwania do zapłaty,Tak,Bank,Nie było wniosku o zabezpieczenie,None,None,None,Wygrana banku,Sąd Okręgowy zmienił wyrok Sądu Rejonowego i o...
2,Tak,None,2013-07-26,Sąd Okręgowy,Sąd I instancji,"art. 28, art. 48 pkt 2, art. 104 ustawy z dnia...",Tak,O zapłatę,Nie,None,...,Tak,Od innej daty,Tak,Powód (bank),Nie było wniosku o zabezpieczenie,None,None,None,Wygrana banku,Powództwo zostało uwzględnione w przeważającej...
3,Tak,białostocka,2014-02-05,Sąd Okręgowy,Sąd odwoławczy,"art. 189 k.p.c., art. 385(1) k.c., art. 69 ust...",Tak,O ustalenie istnienia/nieistnienia stosunku pr...,Nie,None,...,Nie,None,Tak,Pozwany (bank),Nie było wniosku o zabezpieczenie,None,None,None,Wygrana banku,Sąd Apelacyjny oddalił powództwo o ustalenie n...
4,Tak,gdańska,2014-03-11,Sąd Okręgowy,Sąd odwoławczy,"art. 840 § 1 pkt 1 kpc, art. 60 k.c., art. 61 ...",Tak,O ukształtowanie stosunku prawnego,Nie,None,...,None,None,Tak,Powodowie,Nie było wniosku o zabezpieczenie,None,None,None,Wygrana kredytobiorcy,Sąd Apelacyjny zmienił wyrok Sądu Okręgowego i...


In [28]:
print(f"Dataset size: {len(df)}")
print(f"Number of fields: {len(out.columns)}")

Dataset size: 690
Number of fields: 44


# Analysis

In [29]:
from ydata_profiling import ProfileReport

In [30]:
out["sprawa_frankowiczow"].value_counts().reset_index()

,sprawa_frankowiczow,count
0,Tak,659
1,Nie,31


In [32]:
out["apelacja"].value_counts().reset_index()

,apelacja,count
0,warszawska,132
1,białostocka,28
2,katowicka,28
3,poznańska,25
4,gdańska,22
5,szczecińska,20
6,krakowska,17
7,olsztyńska,16
8,łódzka,12
9,wrocławska,4


In [37]:
out["typ_sadu"].value_counts().reset_index()

,typ_sadu,count
0,Sąd Okręgowy,665
1,Sąd Rejonowy,25


In [9]:
# report = ProfileReport(out)
# report.to_file("swiss_franc_loans_annotated.html")